<a href="https://colab.research.google.com/github/cicl-iscl/LeWiDi_SemEval2023/blob/main/notebooks/MTL/MultiLingBert_Romanian_1Head2Neurons_BCE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install transformers
!pip install wandb
# !pip install lightning

# MTL approach
- 1 output neuron for hl and 1 for sl
- Use Soft and Hard Labels (Weight hard loss like it would only be used every 2nd batch)
- Hard:
  - Encoding: ~~1_hot~~ / One label 
  - Loss: ~~Cross_entropy~~/ ~~Negative_Log_Like~~/ BCE_Loss
- Soft
  - Loss: ~~Cross_entropy~~ / ~~KL_div~~ / BCE_Loss

- Model output layer:
  - 1 Layer with 2 outputs (loss with [:1] & [1:])~~
  - ~~2 seperate headers~~
  - ~~1 header and alternating loss (different loss)~~

- Layers to train
 - ~~ Whole model~~
 - output
 - Specific number of last layers: 3

- Regularization:
  - ~~Dropout~~



**Results**: 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import wandb
wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: sheuschk (capture_disagreement). Use `wandb login --relogin` to force relogin


True

In [4]:
from drive.MyDrive.cicl_data.helpers import read_data
# from drive.MyDrive.cicl_data.code import CustomLabelDataset

In [5]:
import json
import numpy as np
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_scheduler
# from datasets import Dataset
import torch.nn.functional as Fun
from torch.utils.data import Dataset, random_split, DataLoader
from torch.optim import AdamW
import torch.nn as nn

from tqdm.notebook import tqdm


In [8]:
seed = 1
torch.manual_seed(seed)
device = "cuda" if torch.cuda.is_available() else "cpu"

In [7]:
data_dict = read_data()
df_all = pd.concat([data_dict[k] for k in data_dict.keys()])

In [9]:
def read_romanian(usage="train"):
  """@usage: 'train'/'dev'"""
  current_file = '/content/drive/MyDrive/cicl_data/romanian/rom_'+ usage +'.json' 
  data = json.load(open(current_file, 'r', encoding = 'UTF-8'))                                   
 
  def extract_soft_labels(row):
    return list(row.values())

  def transform_data(data, name):
    df = pd.DataFrame(data).transpose()
    df = df.astype({"hard_label": int}, errors='raise') 
    df['data_set'] = name
    df["soft_list"] = df["soft_label"].apply(extract_soft_labels)
    return df

  df = transform_data(data, "rom")
  return df

In [10]:
rom_data = read_romanian()
rom_data["split"] = "train"
rom_data.shape

(28254, 12)

In [9]:
# rom_data_small = rom_data.sample(5000, random_state=42)
# rom_data_small["split"] = "train"

In [11]:
df_all = pd.concat([df_all, rom_data])

In [12]:
def extract_soft_labels(row):
  return row[1]

In [13]:
df_all["sl_1s"] = df_all["soft_list"].apply(extract_soft_labels)

### Pretrained model

In [14]:
# Maybe load from wandb in future
tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')

In [15]:
from transformers import BertModel
base_model = BertModel.from_pretrained("bert-base-multilingual-cased")
# class_model = AutoModelForSequenceClassification.from_pretrained("lanwuwei/GigaBERT-v4-Arabic-and-English", num_labels=2)

# output of model: https://huggingface.co/docs/transformers/main_classes/output#transformers.modeling_outputs.BaseModelOutputWithPoolingAndCrossAttentions

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [16]:
class MTLModel(nn.Module):
  def __init__(self, base_model):
    super().__init__()
    self.bert = base_model
    self.linear = nn.Linear(768, 2)
    self.sigmoid = nn.Sigmoid()

  def forward(self, input_ids, attention_mask, token_type_ids):
    """a linear layer on top of the pooled output (https://huggingface.co/transformers/v3.0.2/model_doc/bert.html#bertforsequenceclassification)"""

    x = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
    x = self.linear(x.pooler_output)
    x = self.sigmoid(x)
    return x[:,0].to(dtype=torch.float64), x[:,1].to(dtype=torch.float64)

In [17]:
model = MTLModel(base_model).to(device)

In [18]:
# Deactivate weights for backprob
for name, param in model.named_parameters():
  if "bert.embeddings" in name:
    param.requires_grad = False
  for i in range(0, 9):  # BERT Model has 11 Bert Layers
    if str(i) in name:
      param.requires_grad = False


## Prepare Data

In [ ]:
# from drive.MyDrive.cicl_data.helpers import CustomLabelDataset
# return: input, hard_labels_1hot, soft_labels, hard_labels

In [19]:
class CustomLabelDataset(Dataset):
    def __init__(self, df_all):
        self.text = list(map(self.tokenize_func, df_all["text"]))
        self.soft_labels = df_all["soft_list"] 
        self.hard_labels = df_all["hard_label"]
        self.hard_labels_1h = Fun.one_hot(torch.tensor(df_all['hard_label'].values))
        self.soft_labels_1s = df_all["sl_1s"] # 0.33 of soft labels like {"1": 0.33, "0": 0.67}

    def __len__(self):
        return len(self.text)
      
    def tokenize_func(self, text):
        return tokenizer(text, padding="max_length", truncation=True, max_length=240)

    def __getitem__(self, idx):
        input = {"attention_mask": torch.tensor(self.text[idx]["attention_mask"]),
                 "token_type_ids": torch.tensor(self.text[idx]["token_type_ids"]),
                 "input_ids": torch.tensor(self.text[idx]["input_ids"])}
        return input, self.hard_labels_1h[idx], torch.tensor(self.soft_labels[idx]), torch.tensor(self.hard_labels[idx]), torch.tensor(self.soft_labels_1s[idx])


In [20]:
# Init dataset
dataset = CustomLabelDataset(df_all)
batch_size = 128

train_size = len(dataset)

train_dataloader = DataLoader(
    dataset,
    batch_size=batch_size,
    shuffle=True)


In [21]:
# initialize Evaluation dataset
data_dict_dev = read_data("dev")
df_dev = pd.concat([data_dict_dev[k] for k in data_dict_dev.keys()])

df_dev["sl_1s"] = df_dev["soft_list"].apply(extract_soft_labels)

dev_dataset = CustomLabelDataset(df_dev)
dev_batch_size = 32
dev_size = len(dev_dataset)

dev_dataloader = DataLoader(
    dev_dataset,
    batch_size=dev_batch_size)

## Optimization

In [22]:
# Optimizer
num_epochs = 10

num_training_steps = num_epochs * len(train_dataloader)
optimizer = AdamW(model.parameters())
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [23]:
# Loss
bce_loss = nn.BCELoss()

## Training

In [24]:
run = wandb.init(
    project="MTL",
    config={
        "epochs": num_epochs,
        "batch_size": batch_size,
        "device": device,
        "Seed": seed
        },
      save_code = True,
      tags = ["romanianAll", "bert_multilingual", "MTL", "4_last_layers", "1head_2neurons", "BCE_Loss", "lr_layer"]
      )
wandb.watch(model, log_freq=100)

[]

In [25]:
# from drive.MyDrive.cicl_data.helpers import ce_eval_func
def ce_eval_func(model, eval_dataloader, eval_size, epsilon=1e-12, device="cuda"):
  model.eval()
  cross_error = 0

  for i, batch in enumerate(tqdm(eval_dataloader, 0)):
    input_ids = batch[0]["input_ids"].to(device, dtype=torch.long)
    attention_mask = batch[0]["attention_mask"].to(device, dtype=torch.long)
    token_type_ids = batch[0]["token_type_ids"].to(device, dtype=torch.long)
    soft_labels = batch[2].to(device)

    with torch.no_grad():
      _, pred = model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
    pred = pred.reshape(len(pred), 1)
    probabilities = torch.cat((1-pred, pred), dim=-1)
    # probabilities = torch.softmax(pred, axis=-1)
    
    predictions = torch.clip(probabilities, epsilon, 1. - epsilon)
    cross_error += -torch.sum(soft_labels * torch.log(predictions + 1e-9))

  return cross_error / eval_size



In [26]:
ce_before = ce_eval_func(model, dev_dataloader, dev_size, device=device)
wandb.log({"eval/ce_before_training": ce_before})
print(f"CE before training: {ce_before}")

  0%|          | 0/70 [00:00<?, ?it/s]

CE before training: 0.6909753121830218


In [27]:
# Train
last_ce = 10
smallest_ce = 10
eval_counter = False

for e in range(num_epochs):
  model.train()
  loss_batches = 0
  epoch_loss = 0
  epoch_len = len(train_dataloader)

  for i, batch in enumerate(train_dataloader):
    input_ids = batch[0]["input_ids"].to(device, dtype=torch.long)
    attention_mask = batch[0]["attention_mask"].to(device, dtype=torch.long)
    token_type_ids = batch[0]["token_type_ids"].to(device, dtype=torch.long)
    soft_labels_1 = batch[4].to(device, dtype=torch.float64)
    hard_label = batch[3].to(device, dtype=torch.float64)

    # predict
    optimizer.zero_grad()

    pred_hl, pred_sl = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

    # Loss
    loss = bce_loss(pred_sl, soft_labels_1)
    loss += bce_loss(pred_hl, hard_label) * 0.5

    loss.backward()
    optimizer.step()
    lr_scheduler.step()

    # Log
    loss_batches += loss.item()
    epoch_loss += loss.item()

    log_n_batches = 40
    if i % log_n_batches == 0:
      if i != 0:
        print(f"{e+1}: Last {log_n_batches} batches avg loss: {loss_batches/log_n_batches:>7f}  [{i}/{epoch_len}]")
        wandb.log({"train/loss_over_batches": loss_batches/log_n_batches})
        wandb.log({"train/epochs": e})
      loss_batches = 0
  
  epoch_loss /= i  
  print(f"Epoch [{e+1}/{num_epochs}] mean loss: {epoch_loss:>6f}")
  wandb.log({"train/epoch_loss": epoch_loss})

  # Eval error
  ce = ce_eval_func(model, dev_dataloader, dev_size, device=device)
  print(f"Epoch [{e+1}/{num_epochs}] Eval CE  : {ce:>6f}")
  wandb.log({"eval/epoch_ce": ce})

  # Stop after Eval CE raises 2 times in a row (Simple early stopping)
  if ce > last_ce:
    if eval_counter is True:
      print("Interrupt: Eval Error is raising")
      break;
    eval_counter = True
  elif ce < smallest_ce:
    torch.save(model.state_dict(), 'model.pt')
    print(f"Epoch [{e+1}/{num_epochs}] Save model state")
    eval_counter = False
    smallest_ce = ce
  else:
    eval_counter = False
  
  last_ce = ce



1: Last 40 batches avg loss: 0.614122  [40/303]
1: Last 40 batches avg loss: 0.511019  [80/303]
1: Last 40 batches avg loss: 0.451227  [120/303]
1: Last 40 batches avg loss: 0.478827  [160/303]
1: Last 40 batches avg loss: 0.458328  [200/303]
1: Last 40 batches avg loss: 0.447048  [240/303]
1: Last 40 batches avg loss: 0.439680  [280/303]
Epoch [1/10] mean loss: 0.487799


  0%|          | 0/70 [00:00<?, ?it/s]

Epoch [1/10] Eval CE  : 0.456398
Epoch [1/10] Save model state
2: Last 40 batches avg loss: 0.437333  [40/303]
2: Last 40 batches avg loss: 0.442172  [80/303]
2: Last 40 batches avg loss: 0.402961  [120/303]
2: Last 40 batches avg loss: 0.443253  [160/303]
2: Last 40 batches avg loss: 0.426945  [200/303]
2: Last 40 batches avg loss: 0.414280  [240/303]
2: Last 40 batches avg loss: 0.426453  [280/303]
Epoch [2/10] mean loss: 0.428515


  0%|          | 0/70 [00:00<?, ?it/s]

Epoch [2/10] Eval CE  : 0.441249
Epoch [2/10] Save model state
3: Last 40 batches avg loss: 0.403564  [40/303]
3: Last 40 batches avg loss: 0.402059  [80/303]
3: Last 40 batches avg loss: 0.413554  [120/303]
3: Last 40 batches avg loss: 0.406925  [160/303]
3: Last 40 batches avg loss: 0.387032  [200/303]
3: Last 40 batches avg loss: 0.389603  [240/303]
3: Last 40 batches avg loss: 0.411673  [280/303]
Epoch [3/10] mean loss: 0.403380


...

Epoch [5/10] Eval CE  : 0.431946
Epoch [5/10] Save model state
6: Last 40 batches avg loss: 0.355705  [40/303]
6: Last 40 batches avg loss: 0.358357  [80/303]
6: Last 40 batches avg loss: 0.348483  [120/303]
6: Last 40 batches avg loss: 0.357900  [160/303]
6: Last 40 batches avg loss: 0.362080  [200/303]
6: Last 40 batches avg loss: 0.354302  [240/303]
6: Last 40 batches avg loss: 0.357844  [280/303]
Epoch [6/10] mean loss: 0.358180


  0%|          | 0/70 [00:00<?, ?it/s]

Epoch [6/10] Eval CE  : 0.433063
7: Last 40 batches avg loss: 0.332798  [40/303]
7: Last 40 batches avg loss: 0.336711  [80/303]
7: Last 40 batches avg loss: 0.361592  [120/303]
7: Last 40 batches avg loss: 0.344564  [160/303]
7: Last 40 batches avg loss: 0.345067  [200/303]
7: Last 40 batches avg loss: 0.340775  [240/303]
7: Last 40 batches avg loss: 0.340930  [280/303]
Epoch [7/10] mean loss: 0.343587


  0%|          | 0/70 [00:00<?, ?it/s]

Epoch [7/10] Eval CE  : 0.433851
Interrupt: Eval Error is raising


Model dependent improvements:

## Evaluation

In [28]:
model_best = MTLModel(base_model)
model_best.load_state_dict(torch.load('model.pt'))
model_best = model_best.to(device)

In [29]:
# Final Cross Entropy Error
cross_error = ce_eval_func(model_best, dev_dataloader, dev_size, device=device)
print(f"CE error: {cross_error}")
wandb.log({"dev/ce": cross_error})

  0%|          | 0/70 [00:00<?, ?it/s]

CE error: 0.4319455644790185


In [31]:
# from drive.MyDrive.cicl_data.helpers import f1_eval_func
from sklearn.metrics import f1_score

def f1_eval_func(model, eval_dataloader, eval_size, device="cuda"):
  model.eval()
  f1_error = 0

  for i, batch in enumerate(tqdm(eval_dataloader, 0)):
    input_ids = batch[0]["input_ids"].to(device, dtype = torch.long)
    attention_mask = batch[0]["attention_mask"].to(device, dtype = torch.long)
    token_type_ids = batch[0]["token_type_ids"].to(device, dtype = torch.long)
    hard_labels = batch[3].to(device)

    with torch.no_grad():
      pred, _ = model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
    f1_error += f1_score(hard_labels.cpu(), torch.round(pred).cpu(), average='micro')

  return f1_error / eval_size

In [ ]:
# F1 micro Error
f1 = f1_eval_func(model_best, dev_dataloader, dev_size, device)
print(f"F1 error: {f1}")
wandb.log({"dev/f1": f1})

  0%|          | 0/557 [00:00<?, ?it/s]

F1 error: 0.20696629213483145


In [30]:
model.eval()
cross_error = 0
epsilon = 1e-12
for i, batch in enumerate(dev_dataloader):
  input_ids = batch[0]["input_ids"].to(device, dtype = torch.long)
  attention_mask = batch[0]["attention_mask"].to(device, dtype = torch.long)
  token_type_ids = batch[0]["token_type_ids"].to(device, dtype = torch.long)
  soft_labels = batch[2].to(device)

  with torch.no_grad():
    pred_hl, pred = model_best(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
  pred = pred.reshape(len(pred), 1)
  probabilities = torch.cat((1-pred, pred), dim=-1)
  # probabilities = torch.softmax(pred, axis=-1)
  predictions = torch.clip(probabilities, epsilon, 1. - epsilon)
  cross_error += -torch.sum(soft_labels * torch.log(predictions + 1e-9))
  break



In [31]:
print(predictions)
print(soft_labels)

tensor([[0.6973, 0.3027],
        [0.1262, 0.8738],
        [0.6528, 0.3472],
        [0.4965, 0.5035],
        [0.6662, 0.3338],
        [0.7091, 0.2909],
        [0.7251, 0.2749],
        [0.9434, 0.0566],
        [0.6780, 0.3220],
        [0.6949, 0.3051],
        [0.5697, 0.4303],
        [0.5850, 0.4150],
        [0.4025, 0.5975],
        [0.7016, 0.2984],
        [0.8333, 0.1667],
        [0.7135, 0.2865],
        [0.2945, 0.7055],
        [0.8250, 0.1750],
        [0.6339, 0.3661],
        [0.8527, 0.1473],
        [0.6608, 0.3392],
        [0.8717, 0.1283],
        [0.4405, 0.5595],
        [0.9072, 0.0928],
        [0.7329, 0.2671],
        [0.7323, 0.2677],
        [0.3069, 0.6931],
        [0.3824, 0.6176],
        [0.2938, 0.7062],
        [0.3473, 0.6527],
        [0.4692, 0.5308],
        [0.7054, 0.2946]], device='cuda:0', dtype=torch.float64)
tensor([[0.6700, 0.3300],
        [0.0000, 1.0000],
        [0.3300, 0.6700],
        [0.3300, 0.6700],
        [1.0000, 0.0000],

In [34]:
print(pred_hl)

tensor([0.2687, 0.9226, 0.4211, 0.2404, 0.2646, 0.2663, 0.2679, 0.1216, 0.2659,
        0.2949, 0.2297, 0.4059, 0.3585, 0.2258, 0.3314, 0.1781, 0.7587, 0.1422,
        0.3373, 0.2865, 0.2714, 0.1998, 0.2506, 0.1694, 0.3656, 0.2458, 0.7739,
        0.3999, 0.8225, 0.3163, 0.6996, 0.2546], device='cuda:0',
       dtype=torch.float64)


### Finish

In [32]:
# torch.save(model.state_dict(), 'model.pt')
# model.load_state_dict(torch.load(PATH), strict=False)
artifact = wandb.Artifact(name='model_param', type='model')
artifact.add_file(local_path="model.pt")
run.log_artifact(artifact);

In [33]:
wandb.finish()

dev/ce,▁
eval/ce_before_training,▁
eval/epoch_ce,█▄▂▅▁▁▂
train/epoch_loss,█▅▄▃▃▂▁
train/epochs,▁▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇█████
train/loss_over_batches,█▅▄▅▄▄▄▄▃▃▃▃▃▃▃▂▂▃▂▂▂▃▂▂▂▂▂▂▂▁▂▂▂▁▂▁▂▁▁▁
dev/ce,0.43195
eval/ce_before_training,0.69098
eval/epoch_ce,0.43385
train/epoch_loss,0.34359
train/epochs,6


## TSV files

In [ ]:
import os
import csv

In [ ]:
filepaths = ["/content/ArMIS_results.tsv", "/content/ConvAbuse_results.tsv", "/content/HS-Brexit_results.tsv", "/content/MD-Agreement_results.tsv"]
epsilon = 1e-12

for fp in filepaths:
  if os.path.exists(fp):
    os.remove(fp)

for key in data_dict_dev.keys():
  data_dict_dev[key]["sl_1s"] = data_dict_dev[key]["soft_list"].apply(extract_soft_labels)
  tsv_dataset = CustomLabelDataset(data_dict_dev[key])
  tsv_dataloader = DataLoader(tsv_dataset, shuffle=False, batch_size=1)
  filepath_write = f"/content/{key}_results.tsv"
  
  if "HS-" in key:
    task = "HS"
  elif "MD-" in key:
    task = "MD"
  elif "Conv" in key:
    task = "Abu"
  elif "MIS" in key:
    task = "Mis"

  with open(filepath_write, 'w', newline='') as tsvfile:
      writer = csv.writer(tsvfile, delimiter='\t', lineterminator='\n')
      for i, batch in enumerate(tqdm(tsv_dataloader, 0)):
        input_ids = batch[0]["input_ids"].to(device, dtype = torch.long)
        attention_mask = batch[0]["attention_mask"].to(device, dtype = torch.long)
        token_type_ids = batch[0]["token_type_ids"].to(device, dtype = torch.long)

        with torch.no_grad():
          pred = model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, task=task)
        # logits = pred.logits
        # probability = torch.softmax(pred, axis=-1)
        pred = pred.reshape(len(pred), 1)
        probability = torch.cat((1-pred, pred), dim=-1)
        # probability = torch.softmax(pred, axis=-1)
        prediction = torch.round(pred)
        probability = torch.clip(probability, epsilon, 1. - epsilon) # Really necessary?
        writer.writerow([int(prediction[0].item()), probability[0][0].item(), probability[0][1].item()])


  0%|          | 0/141 [00:00<?, ?it/s]

  0%|          | 0/1104 [00:00<?, ?it/s]

  0%|          | 0/812 [00:00<?, ?it/s]

  0%|          | 0/168 [00:00<?, ?it/s]

In [ ]:
from zipfile import ZipFile

filepath = "res.zip" 

if os.path.exists(filepath):
    os.remove(filepath)

#loop over filepath names throws an string index out of range for whatever reason(also can't use content here, not sure why)
with ZipFile(filepath, 'w') as zipObj:
  zipObj.write("MD-Agreement_results.tsv")
  zipObj.write("ArMIS_results.tsv")
  zipObj.write("HS-Brexit_results.tsv")
  zipObj.write("ConvAbuse_results.tsv")

In [ ]:
from google.colab import files
files.download("res.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>